# Faster_whisperのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用することで高速化できますが，1日に使える時間が制限されているので，Faster-Whisperくらいなら（速度を重視しないなら）cpuインスタンスのまま利用することを推奨します。

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#Faster-Whisper で必要なモジュールのインストール
!pip install faster_whisper
#whisper APIで必要なモジュールのインストール
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [2]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

#OpenAIのAPI_keyを設定する
#設定方法は下記を参照
#https://note.com/npaka/n/n79bb63e17685

from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/**/colab_fasterwhisper_sample/Faster-whisper_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Mounted at /content/drive
/content/drive/MyDrive/docomo/colab_zenn/colab_fasterwhisper_sample
/content/drive/MyDrive/docomo/colab_zenn/colab_fasterwhisper_sample
/content/drive/MyDrive/docomo/colab_zenn/colab_fasterwhisper_sample


In [3]:
#faster-whisperモジュールをimportする
from module.module_whisper import FasterWhisperModel
import time

#whisperAPI用のモジュールをimportする
from openai import OpenAI
# OpenAIクライアントの初期化
client = OpenAI(api_key = api_key)

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [11]:
#モデルの設定を行う。
#cpuとGPUで実行の設定を分けている（処理時間の問題）
#同じ設定にすれば，同じ精度で文字起こしされる

config_text = """
[FasterWhisper]
device = auto
language = ja

gpu_model_type = large-v3
gpu_beam_size = 1
gpu_compute_type = float16

cpu_model_type = small
cpu_beam_size = 1
cpu_compute_type = int8

use_kotoba = False
kotoba_model_type = kotoba-tech/kotoba-whisper-v1.0-faster
chunk_length = 15
condition_on_previous_text = False
"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

下記のファイルの内容を文字起こしします。

In [5]:
#文字起こしするファイルを指定する。下記はサンプルファイル

wav_path = "./inputs/audio_recognition/MANA_yofukashi_QUESTION_007.wav"



In [20]:
#通常のfaster-whisperの利用
whisper = FasterWhisperModel()
print("transcribe start!")
start = time.time()
text = whisper.audioFile2text(wav_path)
print("whisper_time:",time.time()-start)
print(text)

transcribe start!
whisper_time: 0.7156243324279785
高いところに行くけど大丈夫そう?


In [8]:
#モデルの設定を行う。
#cpuとGPUで実行の設定を分けている（処理時間の問題）
#同じ設定にすれば，同じ精度で文字起こしされる

config_text = """
[FasterWhisper]
device = auto
language = ja

gpu_model_type = large-v3
gpu_beam_size = 1
gpu_compute_type = float16

cpu_model_type = medium
cpu_beam_size = 1
cpu_compute_type = int8

use_kotoba = True
kotoba_model_type = kotoba-tech/kotoba-whisper-v1.0-faster
chunk_length = 15
condition_on_previous_text = False
"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

In [22]:
#kotoba-whisperの利用
whisper = FasterWhisperModel()
print("transcribe start!")
start = time.time()
text = whisper.audioFile2text(wav_path)
print("whisper_time:",time.time()-start)
print(text)

transcribe start!
whisper_time: 0.4141805171966553
高いところに行くけど大丈夫そう?


In [23]:
#whisper APIの利用
print("transcribe start!")
start = time.time()
transcription = client.audio.transcriptions.create(model="whisper-1",
                        language="ja",
                        file=open(wav_path, "rb"),
                        response_format="verbose_json",
                        prompt = "こんにちは！今日は、あんまり、元気じゃないかな・・・君はどう？"
                        )
print("whisper_time:",time.time()-start)
print(transcription.text)

transcribe start!
whisper_time: 0.9151504039764404
高いところに行くけど、大丈夫そう?


In [13]:
#cpuでのの利用
whisper = FasterWhisperModel()
print("transcribe start!")
start = time.time()
text = whisper.audioFile2text(wav_path)
print("whisper_time:",time.time()-start)
print(text)

transcribe start!
whisper_time: 5.328901052474976
高いところに行くけど大丈夫そう?
